<!--BOOK_INFORMATION-->
<img align="left" style="width:80px;height:98px;padding-right:20px;" src="https://raw.githubusercontent.com/joe-papa/pytorch-book/main/files/pytorch-book-cover.jpg">

This notebook contains an excerpt from the [PyTorch Pocket Reference](http://pytorchbook.com) book by [Joe Papa](http://joepapa.ai); content is available [on GitHub](https://github.com/joe-papa/pytorch-book).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/joe-papa/pytorch-book/blob/main/04_02_Sentiment_Analysis_with_TorchText.ipynb)

# Chapter 4 - Neural Network Development Reference Designs
# Sentiment Analysis with TorchText

## Data Processing

In [3]:
# For reproducibility
import random
import torch

SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
random_state = random.seed(SEED)
torch.cuda.empty_cache()

device = torch.device("cuda:1" if 
    torch.cuda.is_available() else "cpu")

from torchtext.data.utils import get_tokenizer
tokenizer = get_tokenizer('spacy', language='en_core_web_sm') # <1>


In [4]:
print(torch.cuda.current_device())

torch.cuda.set_device(1) 

0


In [5]:
def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

generate_bigrams([
 'This', 'movie', 'is', 'awesome'])
# out:
# ['This', 'movie', 'is', 'awesome', 'This movie',
#  'movie is', 'is awesome']

['This', 'movie', 'is', 'awesome', 'This movie', 'is awesome', 'movie is']

In [6]:
# import torch
# from torchtext import data
# from torchtext import datasets

# TEXT = data.Field(tokenize = 'spacy',
#                   preprocessing = \
#                     generate_bigrams) # <1>

# LABEL = data.LabelField(dtype = torch.float) # <2>

# train_data, test_data = \
#   datasets.IMDB.splits(TEXT, LABEL) # <3>
# train_data, valid_data = train_data.split(
#     random_state=random_state) # <4>

In [7]:
from torchtext.datasets import IMDB
from torch.utils.data.dataset import random_split

train_iter, test_iter = IMDB(
    split=('train', 'test')) #<1>

train_dataset = list(train_iter) #<2>
test_dataset  = list(test_iter)

num_train = int(len(train_dataset) * 0.70)
train_data, valid_data = \
    random_split(train_dataset, 
        [num_train, 
         len(train_dataset) - num_train]) # <3>

In [8]:
print(len(train_data), len(valid_data), len(test_dataset))
# out:17500 7500 25000

data_index = 21
print(train_data[data_index][0])
# out: (your results may vary)
#   pos

print(train_data[data_index][1])
# out: (your results may vary)
# ['This', 'film', 'moved', 'me', 'beyond', 'comprehension', ...

17500 7500 25000
neg
I missed the beginning but I did see most of it. A friend got it on DVD in the cheap room at FYE.<br /><br />The skits are all very short, and yet most of them are still too long. The majority of them, they seem to have forgotten to have something funny! Quite a lot of racist/sexist/"homophobic" humor in it, skits based on stereotypes, or skits which use racist terms for people.<br /><br />I'm trying to remember anything I thought was funny in it, and I'm having trouble.... The logo for the Tunnel Vision network is a lipsticked mouth with an eyeball in it. The mouth opens and closes over the eye like eyelids. Kind of creepy.<br /><br />What a disappointment. Most of the actors went on to better things, and it's lucky this bomb didn't hold them back.


In [9]:
from torchtext.vocab import GloVe
embed_len = 200
global_vectors = GloVe(name='6B', dim=embed_len)

In [10]:
embeddings = global_vectors.get_vecs_by_tokens(tokenizer("Hello, How are you?"), lower_case_backup=True)

embeddings

tensor([[ 0.2661,  0.2182, -0.1100,  ..., -0.1198, -0.1916, -0.1352],
        [ 0.1765,  0.2921, -0.0021,  ..., -0.2077, -0.2319, -0.1081],
        [ 0.1815,  0.2663,  0.0550,  ...,  0.5375,  0.3151,  0.0162],
        [ 0.0367,  0.1989, -0.0930,  ..., -0.0133, -0.0039,  0.7128],
        [ 0.8540,  0.5715, -0.0237,  ...,  0.3108, -0.2230,  0.2037],
        [ 0.3911,  0.4019, -0.1505,  ..., -0.0348,  0.0798,  0.5031]])

In [11]:
global_vectors.get_vecs_by_tokens("<BOS>")

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])

In [12]:
# this is using the frequencies in the data 

# https://coderzcolumn.com/tutorials/artificial-intelligence/how-to-use-glove-embeddings-with-pytorch#Load-Glove-'42B'-Embeddings

from collections import Counter
from torchtext.vocab import vocab
from torchtext.vocab import build_vocab_from_iterator 
# tokenizer(next(train_iter)[1])

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=['<unk>', '<BOS>', '<EOS>', '<PAD>'])
        
# # create vocabulatory
# counter = Counter()
# for (label, line) in train_iter:
#     counter.update(tokenizer(line))

# # be careful, lower case vocab to call function   
# vocab = vocab(counter, min_freq=10, specials=('<unk>', '<BOS>', '<EOS>', '<PAD>'))
vocab.set_default_index(vocab['<unk>'])  # default index for oov words

# why use vocab?

# change data from str to numeric 




In [13]:
len(vocab), vocab(['here', 'is', 'an', 'example']), vocab.lookup_token(21), vocab['was']

# THE LENGTH IS MUCH LARGER THAN USING COUNTER, WHICH IS 23404

(121069, [164, 11, 45, 493], 'was', 21)

In [14]:
# create embedding for dat

# embed_len = 100

glove_embedding_tensor = torch.zeros(len(vocab),  embed_len).to(device)
 
glove_embedding_tensor[0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:1')

In [15]:

for i in range(len(vocab)):
    try: 
        glove_embedding_tensor[i] = global_vectors.get_vecs_by_tokens(vocab.lookup_token(i))
    #except KeyError as e:
    #    glove_embedding_tensor[i] = torch.normal(0, 1, size=(embed_len,))
    except KeyError:
        glove_embedding_tensor[i] = torch.normal(0, 1, size=(embed_len,))
   

In [16]:
glove_embedding_tensor[len(vocab)-2].sum()

tensor(0., device='cuda:1')

In [17]:
vocab.lookup_token(len(vocab)-2) 

'₤100'

In [18]:
#text_transform = lambda x: [global_vectors.get_vecs_by_tokens('<BOS>')] + [global_vectors.get_vecs_by_tokens(token) for token in tokenizer(x)] + [global_vectors.get_vecs_by_tokens('<EOS>')]

#t1 = text_transform("how are you?")
 
glove_embedding_tensor.shape,glove_embedding_tensor[1232:1234,]

(torch.Size([121069, 200]),
 tensor([[-3.7880e-01,  5.1459e-01, -7.3694e-01, -5.6198e-02,  5.2874e-02,
           1.5662e-01,  2.1436e-01,  3.4249e-01,  1.4240e-01, -9.3278e-02,
           1.6948e-01, -1.1071e-01, -5.6108e-01, -6.0902e-01,  8.6495e-01,
           8.8216e-02,  1.9156e-02, -1.8672e-01, -4.5746e-01, -2.9990e-01,
           7.6822e-01,  1.9607e+00,  2.0502e-01, -2.4833e-01,  7.9986e-02,
           2.2780e-01,  4.6934e-01, -9.3518e-02,  2.9816e-01,  3.3182e-02,
           3.8590e-01,  4.7495e-02, -1.7896e-01, -7.4432e-01, -6.6266e-01,
          -4.5246e-02, -4.2622e-01, -4.5248e-01,  7.1457e-02, -4.4291e-01,
          -4.4779e-01,  4.8310e-03, -7.2398e-01, -1.5956e-01, -3.1243e-01,
          -1.8692e-01,  5.8064e-01,  4.1169e-01,  1.1543e-01,  4.9991e-01,
           3.8175e-02, -4.4620e-01,  6.4993e-01,  1.1448e+00, -2.4783e-01,
          -2.7646e-01, -4.7514e-01,  6.5408e-01, -6.5782e-02, -2.7297e-01,
           2.4473e-03, -1.5304e-01, -5.6501e-01, -3.2696e-01,  1.3107e+0

In [19]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

text_transform = lambda x: [vocab['<BOS>']] + [vocab[token] for token in tokenizer(x)] + [vocab['<EOS>']]
label_transform = lambda x: 1 if x == 'pos' else 0
    
def collate_batch(batch):
   label_list, text_list = [], []
   for (_label, _text) in batch: # each represents one text
        label_list.append(label_transform(_label))
        processed_text = torch.tensor(text_transform(_text))
        text_list.append(processed_text)
   return torch.tensor(label_list, dtype=torch.long).to(device), pad_sequence(text_list,  padding_value=1.0).to(device) # not using batch_first=True,

max_words = 1000
# embed_len = 100

def vectorize_batch(batch):
    Y, X = list(zip(*batch))  # here X is the list of input texts 
    X = [tokenizer(x) for x in X] # list of tokenzied text
    X = [tokens+[""] * (max_words-len(tokens))  if len(tokens)<max_words else tokens[:max_words] for tokens in X] # make same length
    X_tensor = torch.zeros(len(batch), max_words, embed_len) # batch input 
    for i, tokens in enumerate(X):
        X_tensor[i] = global_vectors.get_vecs_by_tokens(tokens)
    return X_tensor.reshape(len(batch), -1), torch.tensor(Y) - 1 ## Subtracted 1 from labels to bring in range [0,1,2,3] from [1,2,3,4]

# use pretrained Glove embdedding 
def vectorize_batch2(batch):
    Y, X = list(zip(*batch)) 
    X = [tokenizer(x) for x in X]
    X = [tokens+[""] * (max_words-len(tokens))  if len(tokens)<max_words else tokens[:max_words] for tokens in X]
    X_tensor = torch.zeros(len(batch), max_words, embed_len)
    for i, tokens in enumerate(X):
        X_tensor[i] = global_vectors.get_vecs_by_tokens(tokens)
    label_list  = []
    for i, _label in enumerate(Y):
        label_list.append(label_transform(_label))
    return torch.tensor(label_list, dtype=torch.long).to(device), X_tensor.reshape(len(batch), -1)## Subtracted 1 from labels to bring in range [0,1,2,3] from [1,2,3,4]

# update data disctionary with glove dictionary 
def vectorize_batch3(batch):
    Y, X = list(zip(*batch)) 
    X = [tokenizer(x) for x in X]
    X = [tokens+[""] * (max_words-len(tokens))  if len(tokens)<max_words else tokens[:max_words] for tokens in X]
    X_tensor = torch.zeros(len(batch), max_words, embed_len)
    for i, tokens in enumerate(X):
        X_tensor[i] = global_vectors.get_vecs_by_tokens(tokens)
    label_list  = []
    for i, _label in enumerate(Y):
        label_list.append(label_transform(_label))
    return torch.tensor(label_list, dtype=torch.long).to(device), X_tensor.reshape(len(batch), -1)## Subtracted 1 from labels to bring in range [0,1,2,3] from [1,2,3,4]


# train_data_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True , collate_fn=collate_batch)

In [20]:


# def collate_batch(batch):
#    label_list, text_list = [], []
#    for (_label, _text) in batch:
#         label_list.append(label_pipeline(_label))
#         processed_text = torch.tensor(text_pipeline(_text))
#         text_list.append(processed_text)
#    return (torch.tensor(label_list, dtype=torch.float64).to(device), 
#           pad_sequence(text_list, 
#                        padding_value=1.0).to(device))

# batch_size = 64    
# def batch_sampler():
#     indices = [(i, len(tokenizer(s[1]))) for i, s in enumerate(train_dataset)]
#     random.shuffle(indices)
#     pooled_indices = []
#     # create pool of indices with similar lengths 
#     for i in rvectorize_batch2dices for current batch
#     for i in range(0, len(pooled_indices), batch_size):
#         yield pooled_indices[i:i + batch_size]

BATCH_SIZE = 64
train_dataloader = DataLoader(dataset=train_data, 
                              batch_size=BATCH_SIZE, 
                              shuffle=True , 
                              collate_fn=collate_batch)
 
                  # collate_fn=collate_batch)
valid_dataloader = DataLoader(valid_data, 
                  batch_size=BATCH_SIZE,
                  shuffle=True, 
                  collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, 
                  batch_size=BATCH_SIZE,
                  shuffle=True, 
                  collate_fn=collate_batch)

In [21]:
trainlabel, trainfeature = next(iter(train_dataloader)) 
trainfeature,trainlabel, len(trainfeature), len(trainlabel), trainfeature.shape, # len is by row


# 1182 is the longest length of the batch 


(tensor([[    1,     1,     1,  ...,     1,     1,     1],
         [   13, 12542,    13,  ...,    68,  2642,   114],
         [   88, 13689,  1859,  ...,    11,     5,   116],
         ...,
         [    1,     1,     1,  ...,     1,     1,     1],
         [    1,     1,     1,  ...,     1,     1,     1],
         [    1,     1,     1,  ...,     1,     1,     1]], device='cuda:1'),
 tensor([0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0,
         0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
         0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1], device='cuda:1'),
 1042,
 64,
 torch.Size([1042, 64]))

## Model Design

In [22]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.conv_0 = nn.Conv2d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[0], embedding_dim))
        
        self.conv_1 = nn.Conv2d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[1], embedding_dim))
        
        self.conv_2 = nn.Conv2d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[2], embedding_dim))
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
    
    
    def forward(self, text):
                
        #text = [batch size, sent len] 
        # cnn expects batch first, but text shape is reversed 
        # text.shape [934, 64], label : [64] 
        embedded  = self.embedding(text) 
        
        embedded = embedded.permute(1, 0, 2)# need to put the batch first 

        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim] , [64,1,962,200]
        
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3)) 
        # conv_0 will output tensor of shape [64, 100, 960, 1], 100 - output chanel from 100  filters
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim = 1))
        # size :[64,300]
        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)   # [64,1]

In [23]:
vocab.lookup_indices(["<PAD>"]), len(vocab)

([3], 121069)

In [24]:
INPUT_DIM = len(vocab)
EMBEDDING_DIM = 200
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = vocab.lookup_indices(["<PAD>"])[0]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [25]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 24,454,401 trainable parameters


In [26]:
model.embedding.weight.data.copy_(glove_embedding_tensor)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.2642,  0.0694, -0.3210,  ..., -0.0317, -0.1033, -0.0825],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [27]:
model.embedding.weight.data.shape

torch.Size([121069, 200])

Not forgetting to zero the initial weights of our unknown and padding tokens.

## Train & Validation

In [28]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = torch.nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [35]:
for epoch in range(5):
  epoch_loss = 0
  epoch_acc = 0
  
  model.train()
  for label, text in train_dataloader:
      #print(label.dtype)
      #print(text)
      optimizer.zero_grad()
      predictions = model(text).squeeze(1)
      #print(predictions.dtype)
      loss = criterion(predictions, label.float())
      
      rounded_preds = torch.round(
          torch.sigmoid(predictions))
      correct = \
        (rounded_preds == label).float()
      acc = correct.sum() / len(correct)
      
      loss.backward()
      optimizer.step()
      epoch_loss += loss.item()
      epoch_acc += acc.item()

  print("Epoch %d Train: Loss: %.4f Acc: %.4f" %
          (epoch,
          epoch_loss / len(train_dataloader), 
          epoch_acc / len(train_dataloader)))

  epoch_loss = 0
  epoch_acc = 0
  model.eval()
  with torch.no_grad():
    for label, text in valid_dataloader:
      predictions = model(text).squeeze(1)
      loss = criterion(predictions, label.float())
      
      rounded_preds = torch.round(
          torch.sigmoid(predictions))
      correct = \
        (rounded_preds == label).float()
      acc = correct.sum() / len(correct)
      
      epoch_loss += loss.item()
      epoch_acc += acc.item()

  print("Epoch %d Valid: Loss: %.4f Acc: %.4f" %
          (epoch,
          epoch_loss / len(valid_dataloader), 
          epoch_acc / len(valid_dataloader)))
  
# out: (your results may vary)
# Epoch 0 Train: Loss: 0.6523 Acc: 0.7165
# Epoch 0 Valid: Loss: 0.5259 Acc: 0.7474
# Epoch 1 Train: Loss: 0.5935 Acc: 0.7765
# Epoch 1 Valid: Loss: 0.4571 Acc: 0.7933
# Epoch 2 Train: Loss: 0.5230 Acc: 0.8257
# Epoch 2 Valid: Loss: 0.4103 Acc: 0.8245
# Epoch 3 Train: Loss: 0.4559 Acc: 0.8598
# Epoch 3 Valid: Loss: 0.3828 Acc: 0.8549
# Epoch 4 Train: Loss: 0.4004 Acc: 0.8813
# Epoch 4 Valid: Loss: 0.3781 Acc: 0.8675


# WHY THE RESULT IS VERY CLOSE TO CREATING VOCAB USING COUNTER
# ALTHOUGH THE SIZE OF VOCAB IS VEYR DIFFERENT 

Epoch 0 Train: Loss: 0.0070 Acc: 0.9985
Epoch 0 Valid: Loss: 0.4509 Acc: 0.8741
Epoch 1 Train: Loss: 0.0036 Acc: 0.9998
Epoch 1 Valid: Loss: 0.4472 Acc: 0.8780
Epoch 2 Train: Loss: 0.0026 Acc: 0.9998
Epoch 2 Valid: Loss: 0.4638 Acc: 0.8776
Epoch 3 Train: Loss: 0.0018 Acc: 0.9999
Epoch 3 Valid: Loss: 0.4828 Acc: 0.8769
Epoch 4 Train: Loss: 0.0014 Acc: 0.9997
Epoch 4 Valid: Loss: 0.4972 Acc: 0.8777


# Testing & Deployment

In [36]:
test_loss = 0
test_acc = 0
model.eval() # <1>
with torch.no_grad(): # <1>
  for label, text in test_dataloader:
    predictions = model(text).squeeze(1)
    loss = criterion(predictions, label.float())
    
    rounded_preds = torch.round(
        torch.sigmoid(predictions))
    correct = \
      (rounded_preds == label).float()
    acc = correct.sum() / len(correct)

    test_loss += loss.item()
    test_acc += acc.item()

print("Test: Loss: %.4f Acc: %.4f" %
        (test_loss / len(test_dataloader), 
        test_acc / len(test_dataloader)))
# out: (your results will vary)
#   Test: Loss: 0.3821 Acc: 0.8599

Test: Loss: 0.5072 Acc: 0.8701


In [37]:
import spacy
nlp = spacy.load('en_core_web_sm')


text_pipeline = lambda x: [vocab[token] 
      for token in tokenizer(x)]


def predict_sentiment(model, sentence):
    model.eval()
    text = torch.tensor(text_pipeline(sentence)).unsqueeze(1).to(device)
    prediction = torch.sigmoid(model(text))
    return prediction.item()

sentiment = predict_sentiment(model, 
                  "Don't waste your time")
print(sentiment)
# out: 4.763594888613835e-34

sentiment = predict_sentiment(model, 
                  "You gotta see this movie!")
print(sentiment)
# out: 0.941755473613739

0.02011590264737606
0.8684893846511841


In [38]:
torch.save(model.state_dict(), 'cnn-model.pt')

In [1]:
torch.cuda.empty_cache()

NameError: name 'torch' is not defined